In [1]:
import pandas as pd

In [38]:
df=pd.read_csv(r'C:\Users\maose\Code\music-util\src\music_util\out\vocals_notes.csv', index_col=0)

In [40]:
df.index.name = ""
df

,pitch,confidence,note.1,chg,not_rest,duration_ms,beat_ms
,,,,,,,
1,2005.55070,0.002940,NaN,4302,False,43020,448.0
2,233.21898,0.775766,A♯3,70,True,700,448.0
3,293.73470,0.521250,C♯4,128,True,1280,448.0
4,330.26242,0.837327,D♯4,124,True,1240,448.0
5,417.48746,0.783543,G♯4,21,True,210,448.0
...,...,...,...,...,...,...,...
201,472.71094,0.797261,B4,74,True,740,448.0
202,435.36313,0.779735,A♯4,101,True,1010,448.0
203,443.85214,0.738357,G♯4,154,True,1540,448.0


In [41]:
df['note']=df['note.1']

In [42]:
df

,pitch,confidence,note.1,chg,not_rest,duration_ms,beat_ms,note
,,,,,,,,
1,2005.55070,0.002940,NaN,4302,False,43020,448.0,NaN
2,233.21898,0.775766,A♯3,70,True,700,448.0,A♯3
3,293.73470,0.521250,C♯4,128,True,1280,448.0,C♯4
4,330.26242,0.837327,D♯4,124,True,1240,448.0,D♯4
5,417.48746,0.783543,G♯4,21,True,210,448.0,G♯4
...,...,...,...,...,...,...,...,...
201,472.71094,0.797261,B4,74,True,740,448.0,B4
202,435.36313,0.779735,A♯4,101,True,1010,448.0,A♯4
203,443.85214,0.738357,G♯4,154,True,1540,448.0,G♯4


In [24]:
min_note_len =448.0 /32

In [45]:
df['num_beats'] = df['duration_ms'] / 448.0

In [26]:
df

,pitch,confidence,note.1,chg,not_rest,duration_ms,num_beats
note,,,,,,,
1,2005.55070,0.002940,NaN,4302,False,43020,96.026786
2,233.21898,0.775766,A♯3,70,True,700,1.562500
3,293.73470,0.521250,C♯4,128,True,1280,2.857143
4,330.26242,0.837327,D♯4,124,True,1240,2.767857
5,417.48746,0.783543,G♯4,21,True,210,0.468750
...,...,...,...,...,...,...,...
201,472.71094,0.797261,B4,74,True,740,1.651786
202,435.36313,0.779735,A♯4,101,True,1010,2.254464
203,443.85214,0.738357,G♯4,154,True,1540,3.437500


In [27]:
valid_notes = pd.Series([1/32,1/16, 20, 100])

In [43]:
import numpy as np
valid_notes = pd.Series([-np.inf,1/16,1/4,1/2,1,2,4,8])
labels = ['32','16','8','4','2','1','1/2']

valid_notes

0      -inf
1    0.0625
2    0.2500
3    0.5000
4    1.0000
5    2.0000
6    4.0000
7    8.0000
dtype: float64

In [47]:
df['note_cat']=pd.cut(df['num_beats'], valid_notes, labels=labels).fillna('1/2')

In [48]:
df

,pitch,confidence,note.1,chg,not_rest,duration_ms,beat_ms,note,num_beats,note_cat
,,,,,,,,,,
1,2005.55070,0.002940,NaN,4302,False,43020,448.0,NaN,96.026786,1/2
2,233.21898,0.775766,A♯3,70,True,700,448.0,A♯3,1.562500,2
3,293.73470,0.521250,C♯4,128,True,1280,448.0,C♯4,2.857143,1
4,330.26242,0.837327,D♯4,124,True,1240,448.0,D♯4,2.767857,1
5,417.48746,0.783543,G♯4,21,True,210,448.0,G♯4,0.468750,8
...,...,...,...,...,...,...,...,...,...,...
201,472.71094,0.797261,B4,74,True,740,448.0,B4,1.651786,2
202,435.36313,0.779735,A♯4,101,True,1010,448.0,A♯4,2.254464,1
203,443.85214,0.738357,G♯4,154,True,1540,448.0,G♯4,3.437500,1


In [53]:
def note2lily(note_cnts):
    notations = []
    for _, r in note_cnts.iterrows():
        if r["not_rest"]:
            linote = r["note"][0].lower()
            octa = r["note"][-1]
            curr_not = linote
            if len(r["note"]) == 3:
                curr_not += 'is'
            if octa == '4':
                curr_not += "'"
            if r["note_cat"] == '1/2':
                notations.append(curr_not + "1")
                notations.append(curr_not + "1")
            else:
                notations.append(curr_not + r["note_cat"] )
        else:
            if r["note_cat"]  == '1/2':
                notations.append('r1')
                notations.append('r1')
            else:
                notations.append('r'+r["note_cat"] )
    return notations
        
nots = note2lily(df)

In [54]:
nots

['r1',
 'r1',
 'ais2',
 "cis'1",
 "dis'1",
 "gis'8",
 "ais'4",
 "gis'1",
 "fis'16",
 "f'1",
 "dis'2",
 'r16',
 'ais2',
 "cis'2",
 'r1',
 "fis'4",
 "f'2",
 'r16',
 'ais2',
 'r4',
 'ais2',
 'gis1',
 'r2',
 'a16',
 'ais4',
 'r4',
 "cis'2",
 "dis'2",
 "f'2",
 "gis'1",
 'r1',
 'r1',
 "cis'1",
 "cis'1",
 'ais2',
 'r2',
 "cis'1",
 "cis'1",
 'r1',
 "gis'1",
 'r2',
 "gis'4",
 "fis'1",
 "cis'8",
 'r2',
 "f'1",
 'r16',
 'ais2',
 'r4',
 "d'1",
 'r1',
 'r1',
 "f'1",
 'c1',
 'cis4',
 'c16',
 'cis16',
 'c8',
 'cis8',
 "fis'1",
 "dis'2",
 "cis'4",
 'r2',
 "dis'4",
 'r8',
 "fis'16",
 "f'2",
 "dis'8",
 "e'4",
 "dis'2",
 "cis'4",
 'r16',
 "e'4",
 "a'4",
 "ais'4",
 'cis4',
 "ais'8",
 'c2',
 "gis'4",
 'r8',
 "e'16",
 "dis'4",
 "f'1",
 "f'1",
 'r2',
 "f'4",
 "g'2",
 'c2',
 'cis1',
 "f'4",
 'r4',
 "f'1",
 "f'1",
 "cis'2",
 "f'1",
 "f'1",
 'ais2',
 'r1',
 'r1',
 'a1',
 'r4',
 "dis'1",
 "a'16",
 "gis'1",
 'r2',
 "dis'1",
 'r2',
 'ais8',
 "cis'2",
 'r1',
 "f'1",
 'r4',
 'ais1',
 "cis'8",
 'gis4',
 'r1',
 'r1',
